In [70]:
import pandas as pd
import numpy as np

In [71]:
ds1 = pd.read_csv('data/21600-0001-Data.tsv',sep='\t')
ds5 = pd.read_csv('data/21600-0005-Data.tsv',sep='\t')
ds8 = pd.read_csv('data/21600-0008-Data.tsv',sep='\t')
ds22 = pd.read_csv('data/21600-0022-Data.tsv',sep='\t')

/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6,12,55,124,125,167,195,196,197,198,336,356,513,522,523,743,745,746,832,833,837,838,839,846,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,956,957,964,969,970,972,976,977,979,981,982,984,1000,1025,1026,1027,1028,1029,1030,1031,1032,1033,1034,1035,1036,1037,1038,1039,1040,1041,1042,1043,1063,1065,1068,1069,1070,1071,1074,1075,1076,1077,1078,1081,1082,1083,1084,1085,1087,1088,1089,1109,1131,1132,1133,1134,1135,1136,1137,1138,1139,1140,1141,1142,1143,1144,1145,1146,1147,1148,1149,1150,1151,1152,1153,1154,1174,1175,1176,1177,1180,1181,1182,1183,1184,1187,1188,1189,1190,1191,1209,1211,1212,1225,1226,1227,1228,1229,1230,1252,1253,1254,1255,1256,1257

In [72]:
df_map = {'ds1': ds1, 'ds5': ds5, 'ds8': ds8, 'ds22': ds22}

time_map = {'ds1': 0, 'ds5': 1, 'ds8': 6, 'ds22': 13}

column_map = {
    'ds1': {
        'H1TO32': 'marijuana_1',
        'H1TO15': 'alcohol_1',
        'H1TO40': 'drugs_1',
        'H1DS2': 'behavior_1',
        'H1GI18': 'attendance_1',
        'H1TO7': 'smoking_1',
        'S1': 'age_1',
        'BIO_SEX': 'sex_1',
        'S6C': 'asian_1',
        'S6B': 'black_1',
        'S6A': 'white_1',
        'S6D': 'nativeamerican_1',
        'S6E': 'other_1'
        #'S4': 'hispanic_1'
    },
    'ds5': {
        'H2TO46': 'marijuana_2',
        'H2TO19': 'alcohol_2',
        'H2TO58': 'drugs_2',
        'H2DS2': 'behavior_2',
        'H2GI6': 'attendance_2',
        'H2TO7': 'smoking_2',
        'CALCAGE2': 'age_2',
        'BIO_SEX2': 'sex_2',
    },
    'ds8': {
        'H3TO110': 'marijuana_3',
        'H3TO38': 'alcohol_3',
        'H3TO117': 'drugs_3',
        'H3DS1': 'behavior_3',
        'H3ED23': 'attendance_3',
        'H3TO10': 'smoking_3',
        'FRIEND': 'age_3', # not actual column
        'BIO_SEX3': 'sex_3',
        'H3OD2': 'hispanic_3'
    },
    'ds22': {
        'H4TO71': 'marijuana_4',
        'H4TO35': 'alcohol_4',
        'H4TO65E': 'drugs_4',
        'H4DS1': 'behavior_4',
        'H4ED6': 'attendance_4',
        'H4TO6': 'smoking_4',
        'PRISON4': 'age_4', # not actual column
        'BIO_SEX4': 'sex_4',
    },
}

In [73]:
dfs = []
new_df_map = {}
for i, (wave, cols) in enumerate(column_map.items()):
    df = df_map[wave]
    df = df[['AID'] + list(cols.keys())]
    df = df.rename(cols, axis=1)
    df = df.set_index('AID')
    df['time_'+str(i+1)] = time_map[wave]
    dfs.append(df)
    new_df_map[wave] = df
    
full_df = pd.concat(dfs, axis=1, join='inner').reset_index()

For small categories, these are missing values
- 6  
- 7 (legitimate skip)
- 8

For medium categories, these are missing values
- 96  
- 97 (legitimate skip)
- 98

For large categories, these are missing values
- 996 
- 997 (legitimate skip)
- 998

In [74]:
# Paper says n=6504, first wave table
new_df_map['ds1'].describe()

,marijuana_1,alcohol_1,behavior_1,attendance_1,smoking_1,sex_1,time_1
count,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.000000,6504.0
mean,745.144834,46.892066,0.291205,0.983702,74.107011,1.516759,0.0
std,431.505517,45.795986,0.851450,0.194604,39.438188,0.502825,0.0
min,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.0
25%,60.000000,5.000000,0.000000,1.000000,30.000000,1.000000,0.0
50%,997.000000,7.000000,0.000000,1.000000,97.000000,2.000000,0.0
75%,997.000000,97.000000,0.000000,1.000000,97.000000,2.000000,0.0
max,999.000000,98.000000,9.000000,8.000000,98.000000,6.000000,0.0


In [75]:
# Need to fix some broken data here
new_df_map['ds1']['drugs_1'] = pd.to_numeric(new_df_map['ds1']['drugs_1'], errors='coerce')

In [76]:
# Check some stats from table
new_df_map['ds1'].loc[new_df_map['ds1']['marijuana_1'] > 995, 'marijuana_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['alcohol_1'] > 95, 'alcohol_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['drugs_1'] > 95, 'drugs_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['attendance_1'] > 5, 'attendance_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['behavior_1'] > 5, 'behavior_1'] = np.NaN
new_df_map['ds1'].loc[new_df_map['ds1']['smoking_1'] > 95, 'smoking_1'] = np.NaN
new_df_map['ds1'].describe()
# Yay, these all align!

,marijuana_1,alcohol_1,drugs_1,behavior_1,attendance_1,smoking_1,sex_1,time_1
count,1655.000000,3545.000000,6402.000000,6443.000000,6501.000000,1653.000000,6504.000000,6504.0
mean,7.219940,5.067419,1.103405,0.229241,0.980772,6.921355,1.516759,0.0
std,35.201338,1.493296,3.899566,0.559196,0.137335,8.178834,0.502825,0.0
min,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0
25%,0.000000,4.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.0
50%,1.000000,5.000000,0.000000,0.000000,1.000000,4.000000,2.000000,0.0
75%,4.000000,6.000000,0.000000,0.000000,1.000000,10.000000,2.000000,0.0
max,800.000000,7.000000,18.000000,3.000000,1.000000,89.000000,6.000000,0.0


In [77]:
# Only 3342 consistent IDs throughout experiment?
full_df

,AID,marijuana_1,alcohol_1,drugs_1,behavior_1,attendance_1,smoking_1,age_1,sex_1,asian_1,...,time_3,marijuana_4,alcohol_4,drugs_4,behavior_4,attendance_4,smoking_4,age_4,sex_4,time_4
0,57101310,997,5,0,0,1,8,17,2,0,...,6,97,2,0,0,0,4,0,2,13
1,57103869,998,97,98,0,1,1,,1,,...,6,1,97,0,0,0,2,0,1,13
2,57109625,0,5,0,1,1,97,13,1,0,...,6,6,5,1,0,0,997,0,1,13
3,57111071,997,97,0,0,1,97,13,1,0,...,6,97,4,1,0,0,997,0,1,13
4,57113943,997,97,0,0,1,97,15,1,0,...,6,0,1,0,0,0,997,0,1,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3337,99718925,997,97,0,0,1,97,14,1,0,...,6,97,97,0,0,0,20,0,1,13
3338,99718944,997,97,0,0,1,97,13,1,0,...,6,97,4,0,0,1,997,0,1,13
3339,99719378,997,97,0,0,1,97,,2,,...,6,97,4,0,0,0,4,0,2,13
3340,99719930,997,97,0,0,1,97,12,2,0,...,6,97,1,0,0,0,997,0,2,13


In [78]:
# List of AIDs
AIDs = list(full_df['AID'])

In [79]:
# Ages of participants
ages = full_df[['AID', 'age_1']]
ages.rename({'age_1':'age'}, axis=1, inplace=True)
ages.set_index('AID', inplace=True)

/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/pandas/core/frame.py:5039: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [80]:
# Hispanic identity of participants
hispanic = full_df[['AID', 'hispanic_3']]
hispanic.rename({'hispanic_3':'hispanic'}, axis=1, inplace=True)
hispanic.set_index('AID', inplace=True)

## Constructing data as constructed by (Adzrago et. al)

It seems clear that each datapoint was independent of others when fed into the linear model.

Need to filter out AIDs that were not continous through entire study...

In [81]:
dfs = []
new_df_map = {}
for i, (wave, cols) in enumerate(column_map.items()):
    df = df_map[wave]
    df = df[['AID'] + list(cols.keys())]
    
    # Here, we remove the numbering
    new_col_names = {}
    for k,v in cols.copy().items():
        new_col_names[k] = v.split('_')[0]
    
    df = df.rename(new_col_names, axis=1)
    df['time'] = time_map[wave]

    # Filter AIDs here
    df = df[df['AID'].isin(AIDs)]

    # Add proper age
    df.set_index('AID', inplace=True)
    age = 'age'+str(i)
    ages_here = ages.rename({'age':age}, axis=1)
    ages_here[age] = pd.to_numeric(ages_here[age], errors="coerce")
    ages_here.loc[ages_here[age].notna(), age] += time_map[wave]
    df = pd.concat([df, ages_here], axis=1, join='inner')
    
    # Assign hispanic
    h = 'hispanic'+str(i)
    hispanic_here = hispanic.rename({'hispanic':h}, axis=1)
    df = pd.concat([df, hispanic_here], axis=1, join='inner')

    df.reset_index(inplace=True)
    
    dfs.append(df)
    new_df_map[wave] = df

full_df = pd.concat(dfs, axis=0, ignore_index=True)#.reset_index()

# Fix age columns
full_df['full_age'] = 0
age_cols = ['age0','age1','age2','age3']
full_df['full_age'] = full_df[age_cols].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)
full_df['full_age'] = pd.to_numeric(full_df['full_age'], errors="coerce")
full_df = full_df.drop(age_cols+['age'],axis=1)
full_df.rename({'full_age':'age'}, axis=1, inplace=True)

# Fix hispanic column
hispanic_cols = ['hispanic0','hispanic1','hispanic2','hispanic3']
full_df['full_hispanic'] = full_df[hispanic_cols].apply(
    lambda x: ''.join(x.dropna().astype(str)),
    axis=1
)
full_df = full_df.drop(hispanic_cols+['hispanic'],axis=1)
full_df.rename({'full_hispanic':'hispanic'}, axis=1, inplace=True)

# Fix race columns
race_cols = ['asian','black','white','nativeamerican','other']
race = dfs[0][['AID'] + race_cols]
race.set_index('AID', inplace=True)

for i, row in full_df.iterrows():
    for c in race_cols:
        full_df.at[i,c] = race.loc[row['AID'],c]


In [84]:
# Need to fix some broken data here
full_df['drugs'] = pd.to_numeric(full_df['drugs'], errors='coerce')
full_df['marijuana'] = pd.to_numeric(full_df['marijuana'], errors='coerce')
full_df['alcohol'] = pd.to_numeric(full_df['alcohol'], errors='coerce')
full_df['attendance'] = pd.to_numeric(full_df['attendance'], errors='coerce')
full_df['behavior'] = pd.to_numeric(full_df['behavior'], errors='coerce')
full_df['smoking'] = pd.to_numeric(full_df['smoking'], errors='coerce')

# Fix missing vals
full_df.loc[full_df['marijuana'] > 995, 'marijuana'] = np.NaN
full_df.loc[full_df['alcohol'] > 95, 'alcohol'] = np.NaN
full_df.loc[full_df['drugs'] > 95, 'drugs'] = np.NaN
full_df.loc[full_df['attendance'] > 5, 'attendance'] = np.NaN
full_df.loc[full_df['behavior'] > 5, 'behavior'] = np.NaN
full_df.loc[full_df['smoking'] > 95, 'smoking'] = np.NaN

In [90]:
full_df

,AID,marijuana,alcohol,drugs,behavior,attendance,smoking,sex,asian,black,white,nativeamerican,other,time,age,hispanic
0,57101310,NaN,5.0,0.0,0.0,1.0,8.0,2,0,1,0,0,0,0,17.0,0.0
1,57103869,NaN,NaN,NaN,0.0,1.0,1.0,1,,,,,,0,NaN,0.0
2,57109625,0.0,5.0,0.0,1.0,1.0,NaN,1,0,0,0,0,1,0,13.0,0.0
3,57111071,NaN,NaN,0.0,0.0,1.0,NaN,1,0,0,1,0,0,0,13.0,0.0
4,57113943,NaN,NaN,0.0,0.0,1.0,NaN,1,0,0,0,0,1,0,15.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13363,99718925,97.0,NaN,0.0,0.0,0.0,20.0,1,0,0,1,0,0,13,27.0,0.0
13364,99718944,97.0,4.0,0.0,0.0,1.0,NaN,1,0,0,1,0,0,13,26.0,0.0
13365,99719378,97.0,4.0,0.0,0.0,0.0,4.0,2,,,,,,13,NaN,0.0
13366,99719930,97.0,1.0,0.0,0.0,0.0,NaN,2,0,1,0,0,0,13,25.0,0.0


## Starting to replicate numbers from the paper
Now that we have the data in order, we need to start checking if we can replicate numbers from the paper's model.

TODO: Reformat below to build a model with all covariates, not just one:

https://www.statsmodels.org/dev/generated/statsmodels.regression.mixed_linear_model.MixedLM.html

In [95]:
# Test
smoking_time_sex = full_df[['smoking','time','sex']].dropna()
smoking_time_sex

,smoking,time,sex
0,8.0,0,2
1,1.0,0,1
8,1.0,0,1
9,1.0,0,2
10,3.0,0,2
...,...,...,...
13360,13.0,13,1
13361,4.0,13,2
13363,20.0,13,1
13365,4.0,13,2


In [96]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
model = 'smoking ~ time'
md = smf.mixedlm(model, data=smoking_time_sex, groups=smoking_time_sex["sex"])
mdf = md.fit()
print(mdf.summary())

          Mixed Linear Model Regression Results
Model:            MixedLM Dependent Variable: smoking    
No. Observations: 4178    Method:             REML       
No. Groups:       2       Scale:              69.4746    
Min. group size:  2006    Log-Likelihood:     -14793.0105
Max. group size:  2172    Converged:          Yes        
Mean group size:  2089.0                                 
----------------------------------------------------------
            Coef.  Std.Err.    z     P>|z|  [0.025  0.975]
----------------------------------------------------------
Intercept   6.955     1.440   4.828  0.000   4.132   9.778
time        0.309     0.025  12.479  0.000   0.261   0.358
Group Var   4.078                                         



/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/lucasrosenblatt/opt/miniconda3/envs/heuristic_fairness/lib/python3.8/site-packages/statsmodels/regression/mixed_linear_model.py:2261: ConvergenceWarning: The Hessian matrix at the estimated parameter values is not positive definite.
  warnings.warn(msg, ConvergenceWarning)
